# CoastSat Web: Extracción de información

## Initial Settings

Se realizan todos los imports necesarios para analizar la información de la página web de CoastSat

In [6]:
import os
import geopandas as gpd
import pandas as pd

Diccionario con toda la información del país, de sus regiones y de las playas de cada región. `region_name` es la región que se analizará y la carpeta CoastSat Web almacenará la información

In [7]:
country_info = {
    "country": "CHILE",
    "country_code": "chi",
    "region": {
        "OHIGGINS": {
            "start_code": "0345",
            "end_code": "0362"
        }
    }
}

region_name = "OHIGGINS"  # Región a procesar
geojson_folder = 'CoastSat_Web'  # Carpeta con los .geojson


## Visualización de la Información

Para una lectura amigable, se convierten los archivos `.geojson` en archivos de excel

In [8]:
def convert_geojson_to_excel(geojson_folder):
    """
    Convierte todos los archivos .geojson de una carpeta a formato .xlsx y los guarda en otra carpeta.

    :param geojson_folder: Ruta de la carpeta que contiene los archivos .geojson.
    """

    # Iterar sobre todos los archivos en la carpeta de entrada
    for file_name in os.listdir(geojson_folder):
        if file_name.endswith('.geojson'):
            # Ruta completa del archivo de entrada
            file_path = os.path.join(geojson_folder, file_name)

            # Cargar el archivo .geojson
            data = gpd.read_file(file_path)

            # Convertir geometría a formato WKT
            data['geometry'] = data['geometry'].apply(lambda x: x.wkt)

            # Ruta completa del archivo de salida
            output_path = os.path.join(geojson_folder, file_name.replace('.geojson', '.xlsx'))

            # Guardar en formato Excel
            data.to_excel(output_path, index=False)
            print(f"Archivo convertido y guardado en: {output_path}")

    print("Todos los archivos .geojson han sido procesados.")

Se convierten los archivos descargados .geoson a un formato amigable para lectura, excel.

In [9]:
convert_geojson_to_excel(geojson_folder)

C:\Users\pedro\AppData\Local\Temp\ipykernel_14472\4100923213.py:18: UserWarning: Geometry column does not contain geometry.
  data['geometry'] = data['geometry'].apply(lambda x: x.wkt)


Archivo convertido y guardado en: CoastSat_Web\CoastSat_polygon_layer.xlsx


C:\Users\pedro\AppData\Local\Temp\ipykernel_14472\4100923213.py:18: UserWarning: Geometry column does not contain geometry.
  data['geometry'] = data['geometry'].apply(lambda x: x.wkt)


Archivo convertido y guardado en: CoastSat_Web\CoastSat_transect_layer.xlsx
Todos los archivos .geojson han sido procesados.


## Extracción de la Región

Luego, el código se centra en una región en específico, definida anteriormente. Se genera un archivo `.xlsx` con la información de los polígonos y los transectos de la región. Además, se crea una carpeta para almacenar esto

In [10]:
def extract_region(geojson_file, country_info, region_name, excel_file):
    """
    Extrae las filas correspondientes a una región de un país en base a un diccionario
    y guarda el resultado en un archivo Excel.

    :param geojson_file: Ruta al archivo .geojson.
    :param country_info: Diccionario con la información del país y las regiones.
    :param region_name: Nombre de la región a filtrar (clave en el diccionario).
    :param excel_file: Ruta para guardar el archivo Excel filtrado.
    """
    # Obtener información del país y región
    country_code = country_info["country_code"]
    region_info = country_info["region"][region_name]
    start_code = int(region_info["start_code"])
    end_code = int(region_info["end_code"])

    # Cargar el archivo GeoJSON
    data = gpd.read_file(geojson_file)



    # Filtrar las filas por el país y el rango de códigos
    filtered_data = data[data['SiteId'].str.startswith(country_code) & 
                         data['SiteId'].str[-4:].astype(int).between(start_code, end_code)]

    # Convertir geometría a formato WKT para exportar a Excel
    filtered_data['geometry'] = filtered_data['geometry'].apply(lambda x: x.wkt)

    # Guardar el resultado en un archivo Excel
    filtered_data.to_excel(excel_file, index=False)
    print(f"Datos filtrados de {region_name} guardados en: {excel_file}")

In [11]:
os.makedirs(os.path.join(os.getcwd(), geojson_folder, region_name), exist_ok=True)
region_folder = os.path.join(os.getcwd(), geojson_folder, region_name)

polygon_file = os.path.join(os.getcwd(), geojson_folder, 'CoastSat_polygon_layer.geojson')
transects_file = os.path.join(os.getcwd(), geojson_folder, 'CoastSat_transect_layer.geojson')

polygon_excel_file = os.path.join(os.getcwd(), region_folder, 'OHIGGINS_polygons.xlsx')
transects_excel_file = os.path.join(os.getcwd(), region_folder, 'OHIGGINS_transects.xlsx')

extract_region(polygon_file, country_info, region_name, polygon_excel_file)
extract_region(transects_file, country_info, region_name, transects_excel_file)


C:\Users\pedro\AppData\Local\Temp\ipykernel_14472\2601581388.py:27: UserWarning: Geometry column does not contain geometry.
  filtered_data['geometry'] = filtered_data['geometry'].apply(lambda x: x.wkt)
c:\Users\pedro\miniforge3\envs\coastsat\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


Datos filtrados de OHIGGINS guardados en: c:\Users\pedro\Documents\RAFA\CoastSat-RRG\CoastSat_Web\OHIGGINS\OHIGGINS_polygons.xlsx
Datos filtrados de OHIGGINS guardados en: c:\Users\pedro\Documents\RAFA\CoastSat-RRG\CoastSat_Web\OHIGGINS\OHIGGINS_transects.xlsx


C:\Users\pedro\AppData\Local\Temp\ipykernel_14472\2601581388.py:27: UserWarning: Geometry column does not contain geometry.
  filtered_data['geometry'] = filtered_data['geometry'].apply(lambda x: x.wkt)
c:\Users\pedro\miniforge3\envs\coastsat\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


## Extracción de Polígonos

Luego, se transforma la información del excel en archivos individuales `.geojson` por playa que contienen los polígonos, que se almacenarán en la misma carpeta de la región

In [12]:
def save_beach_as_geojson(region_excel_file):
    """
    Lee un archivo Excel de playas y guarda cada playa como un archivo .geojson individual,
    creando una subcarpeta para cada SiteId dentro de la carpeta de la región.

    :param region_excel_file: Ruta del archivo Excel que contiene las playas.
    """

    # Cargar el archivo Excel
    data = pd.read_excel(region_excel_file)

    # Crear la carpeta de salida si no existe
    os.makedirs(os.path.join(os.getcwd(), geojson_folder, region_name), exist_ok=True)
    region_folder = os.path.join(os.getcwd(), geojson_folder, region_name)

    # Convertir el DataFrame a un GeoDataFrame
    gdf = gpd.GeoDataFrame(data, geometry=gpd.GeoSeries.from_wkt(data['geometry']))

    # Procesar cada fila (playa)
    for index, row in gdf.iterrows():
        site_id = row['SiteId']
        geometry = row['geometry']

        # Crear un GeoDataFrame con una sola fila
        playa_gdf = gpd.GeoDataFrame([row], geometry=[geometry])

        # Crear la carpeta para el SiteId si no existe
        site_folder = os.path.join(region_folder, site_id)
        os.makedirs(site_folder, exist_ok=True)

        # Ruta completa del archivo .geojson
        output_file = os.path.join(site_folder, f"{site_id}_polygon.geojson")

        # Guardar el archivo .geojson
        playa_gdf.to_file(output_file, driver="GeoJSON")
        print(f"Archivo .geojson guardado para {site_id}: {output_file}")

    print(f"Todos los archivos .geojson han sido guardados en subcarpetas dentro de: {region_folder}")


In [13]:
save_beach_as_geojson(polygon_excel_file)

Archivo .geojson guardado para chi0345: c:\Users\pedro\Documents\RAFA\CoastSat-RRG\CoastSat_Web\OHIGGINS\chi0345\chi0345_polygon.geojson
Archivo .geojson guardado para chi0346: c:\Users\pedro\Documents\RAFA\CoastSat-RRG\CoastSat_Web\OHIGGINS\chi0346\chi0346_polygon.geojson
Archivo .geojson guardado para chi0347: c:\Users\pedro\Documents\RAFA\CoastSat-RRG\CoastSat_Web\OHIGGINS\chi0347\chi0347_polygon.geojson
Archivo .geojson guardado para chi0348: c:\Users\pedro\Documents\RAFA\CoastSat-RRG\CoastSat_Web\OHIGGINS\chi0348\chi0348_polygon.geojson
Archivo .geojson guardado para chi0349: c:\Users\pedro\Documents\RAFA\CoastSat-RRG\CoastSat_Web\OHIGGINS\chi0349\chi0349_polygon.geojson
Archivo .geojson guardado para chi0350: c:\Users\pedro\Documents\RAFA\CoastSat-RRG\CoastSat_Web\OHIGGINS\chi0350\chi0350_polygon.geojson
Archivo .geojson guardado para chi0351: c:\Users\pedro\Documents\RAFA\CoastSat-RRG\CoastSat_Web\OHIGGINS\chi0351\chi0351_polygon.geojson
Archivo .geojson guardado para chi0352: c

c:\Users\pedro\miniforge3\envs\coastsat\Lib\site-packages\pyogrio\geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
c:\Users\pedro\miniforge3\envs\coastsat\Lib\site-packages\pyogrio\geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
c:\Users\pedro\miniforge3\envs\coastsat\Lib\site-packages\pyogrio\geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
c:\Users\pedro\miniforge3\envs\coastsat\Lib\site-packages\pyogrio\geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
c:\Users\pedro\miniforge3\envs\coastsat\Lib\site-packages\py

## Extracción de Transectos

Luego, se transforma la información del excel en archivos individuales `.geojson` por playa que contienen los transectos de cada playa, que se almacenarán en la misma carpeta de la región en subcarpetas por playa

In [14]:
def save_transects_by_siteid(region_excel_file, region_folder):
    """
    Agrupa las filas del Excel por SiteId y guarda cada grupo como un archivo .geojson individual
    en subcarpetas correspondientes dentro de la carpeta de la región.

    :param region_excel_file: Ruta del archivo Excel que contiene los datos de transectos.
    :param region_folder: Carpeta base de salida para los archivos GeoJSON agrupados por SiteId.
    """
    # Crear la carpeta de la región si no existe
    os.makedirs(region_folder, exist_ok=True)

    # Cargar el archivo Excel
    data = pd.read_excel(region_excel_file)

    # Convertir el DataFrame a un GeoDataFrame
    gdf = gpd.GeoDataFrame(data, geometry=gpd.GeoSeries.from_wkt(data['geometry']))

    # Obtener los valores únicos de SiteId
    unique_siteids = gdf['SiteId'].unique()

    # Procesar cada SiteId
    for site_id in unique_siteids:
        # Filtrar las filas correspondientes al SiteId actual
        site_gdf = gdf[gdf['SiteId'] == site_id]

        # Crear la carpeta para el SiteId si no existe
        site_folder = os.path.join(region_folder, site_id)
        os.makedirs(site_folder, exist_ok=True)

        # Ruta completa del archivo .geojson
        output_file = os.path.join(site_folder, f"{site_id}_transects.geojson")

        # Guardar el GeoDataFrame filtrado en un archivo .geojson
        site_gdf.to_file(output_file, driver="GeoJSON")
        print(f"Archivo .geojson guardado para {site_id}: {output_file}")

    print(f"Archivos .geojson han sido guardados en subcarpetas dentro de: {region_folder}")


In [15]:
save_transects_by_siteid(transects_excel_file, region_folder)

Archivo .geojson guardado para chi0345: c:\Users\pedro\Documents\RAFA\CoastSat-RRG\CoastSat_Web\OHIGGINS\chi0345\chi0345_transects.geojson
Archivo .geojson guardado para chi0346: c:\Users\pedro\Documents\RAFA\CoastSat-RRG\CoastSat_Web\OHIGGINS\chi0346\chi0346_transects.geojson
Archivo .geojson guardado para chi0347: c:\Users\pedro\Documents\RAFA\CoastSat-RRG\CoastSat_Web\OHIGGINS\chi0347\chi0347_transects.geojson
Archivo .geojson guardado para chi0348: c:\Users\pedro\Documents\RAFA\CoastSat-RRG\CoastSat_Web\OHIGGINS\chi0348\chi0348_transects.geojson
Archivo .geojson guardado para chi0349: c:\Users\pedro\Documents\RAFA\CoastSat-RRG\CoastSat_Web\OHIGGINS\chi0349\chi0349_transects.geojson
Archivo .geojson guardado para chi0350: c:\Users\pedro\Documents\RAFA\CoastSat-RRG\CoastSat_Web\OHIGGINS\chi0350\chi0350_transects.geojson
Archivo .geojson guardado para chi0351: c:\Users\pedro\Documents\RAFA\CoastSat-RRG\CoastSat_Web\OHIGGINS\chi0351\chi0351_transects.geojson
Archivo .geojson guardado p

c:\Users\pedro\miniforge3\envs\coastsat\Lib\site-packages\pyogrio\geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
c:\Users\pedro\miniforge3\envs\coastsat\Lib\site-packages\pyogrio\geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
c:\Users\pedro\miniforge3\envs\coastsat\Lib\site-packages\pyogrio\geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
c:\Users\pedro\miniforge3\envs\coastsat\Lib\site-packages\pyogrio\geopandas.py:662: UserWarning: 'crs' was not provided.  The output dataset will not have projection information defined and may not be usable in other systems.
  write(
c:\Users\pedro\miniforge3\envs\coastsat\Lib\site-packages\py

In [16]:
carpeta = os.path.join(os.getcwd(), region_folder, 'chi0356')
convert_geojson_to_excel(carpeta)

Archivo convertido y guardado en: c:\Users\pedro\Documents\RAFA\CoastSat-RRG\CoastSat_Web\OHIGGINS\chi0356\chi0356_polygon.xlsx
Archivo convertido y guardado en: c:\Users\pedro\Documents\RAFA\CoastSat-RRG\CoastSat_Web\OHIGGINS\chi0356\chi0356_transects.xlsx
Todos los archivos .geojson han sido procesados.


C:\Users\pedro\AppData\Local\Temp\ipykernel_14472\4100923213.py:18: UserWarning: Geometry column does not contain geometry.
  data['geometry'] = data['geometry'].apply(lambda x: x.wkt)
C:\Users\pedro\AppData\Local\Temp\ipykernel_14472\4100923213.py:18: UserWarning: Geometry column does not contain geometry.
  data['geometry'] = data['geometry'].apply(lambda x: x.wkt)
